# **OCR Pruebas**

In [ ]:
import pytesseract
from pdf2image import convert_from_path

import os

In [9]:
def extract_text(directory_path):
    pdf_texts = {}
    for filename in os.listdir(directory_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(directory_path, filename)
            print(f"Processing: {pdf_path}")
            pages = convert_from_path(pdf_path)
            text = ""
            for page in pages:
                text += pytesseract.image_to_string(page) + "\n"
            pdf_texts[filename] = text
    return pdf_texts

In [10]:
directory = '.\Contratos'
texts = extract_text(directory)
for filename, content in texts.items():
    print(f"--- {filename} ---")
    print(content)

Processing: .\Contratos\C01_Contrato_Prueba-01.pdf


<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Ame Contreras\AppData\Local\Temp\ipykernel_41244\2057534081.py:1: SyntaxWarning: invalid escape sequence '\C'
  directory = '.\Contratos'
C:\Users\Ame Contreras\AppData\Local\Temp\ipykernel_41244\2057534081.py:1: SyntaxWarning: invalid escape sequence '\C'
  directory = '.\Contratos'


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [2]:
import os
from PyPDF2 import PdfReader

def extract_text(directory_path):
    pdf_texts = {}
    for filename in os.listdir(directory_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(directory_path, filename)
            try:
                with open(pdf_path, 'rb') as file:
                    reader = PdfReader(file)
                    text = ''
                    for page in reader.pages:
                        text += page.extract_text()
                pdf_texts[filename] = text
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    return pdf_texts

# Usage example:
directory = '.\\Contratos'
texts = extract_text(directory)
for filename, content in texts.items():
    print(f"--- {filename} ---")
    print(content)

unknown widths : 
[0, IndirectObject(218, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(222, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(86, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(89, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(92, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(95, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(98, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(101, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(104, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(107, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(110, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(113, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(116, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(119, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(122, 0, 1656392107072)]
unknown widths : 
[0, IndirectObject(125, 0, 1656392107072)]
unknown widths : 
[0, Indirec

--- C01_Contrato_Prueba-01.pdf ---
San Juan del Río, Querétaro Página I de 7 
CONTRATO DE ARRENDAMIENTO QUE CELEBRAN, POR UNA PARTE, EL SR. JUAN 
ALEGRIA QUINTANAR, EN LO SUCESIVO EL "ARRENDADOR", CON LA COMPARECENCIA DE LA 
SRA. MARIA; Y POR LA OTRA PARTE, "COMERCIAL IAC, .A. DE C.V.", EN LO SUCESIVO EL 
"ARRENDATARIO", REPRESENTADO EN ESTE ACTO POR SU APODERADO LEGAL EL SR 
VERTICHE, QUIENES SE SUJETAN AL TENOR DE A SIGUIENT ES: 
DECLARACIONES
1.-Del "ARRENDADOR": 
a)Ser persona fisica, dedicada al comercio, mayor de edad y con la plena capacidad jw·ídica para contratar y 
obligarse en los términos del presente contrato.
b)Los señores María y Juan, son designados por el Arrendador como beneficiarios  del pago de rentas cada 
uno al 50%; quienes se obligan a emitir los recibos fisca s correspondientes, tributan bajo el Registro 
Federal de Contribuyentes QUMJ370201FI0, con domicilio tiscal en Ray No. 22, Colonia Centro, C.P. 76800, 
San Juan del Río, Querétaro; y AERJ950629SS5 con dom